In [4]:
import os
import sys
from os.path import join
import json

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import nfl_data_py as nfl

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(ROOT_DIR,'py'))

import util
from plot.plot_simple import plot_play_with_speed

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

with open("paths.json", 'r') as f:
    paths = json.load(f)

PROCESSED_DATA_PATH = paths['processed_data']

In [5]:
WEEK = 2

df_tracking = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'tracking_final.pkl'))
df_game = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'games_final.pkl'))
df_play = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'play_final.pkl'))
df_player_play = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{WEEK}', 'player_play_final.pkl'))
df_player = pd.read_pickle(join(PROCESSED_DATA_PATH, 'players.pkl'))
df_team = pd.read_pickle(join(PROCESSED_DATA_PATH, 'teams.pkl'))

# Drop plays where QB is the ball carrier

In [6]:
(
    df_player_play
    [['game_play_id','had_rush_attempt','nfl_id']]
    .query('had_rush_attempt == 1')
    .merge(df_player[['nfl_id','position']], on='nfl_id', how='left')
    .position.value_counts()
)

RB    552
WR     19
FB      4
QB      2
Name: position, dtype: int64

In [7]:
gpids_with_qb_rusher = (
    df_player_play
    [['game_play_id','had_rush_attempt','nfl_id']]
    .query('had_rush_attempt == 1')
    .merge(df_player[['nfl_id','position']], on='nfl_id', how='left')
    .query('position == "QB"')
    .game_play_id
    .unique()
    .tolist()
)
keep_gids = df_play.query('~game_play_id.isin(@gpids_with_qb_rusher)').game_id.unique().tolist()

if len(gpids_with_qb_rusher) != 0:
    print(f'Dropping {len(gpids_with_qb_rusher)} plays with QB rusher')
    df_tracking = df_tracking.query('~game_play_id.isin(@gpids_with_qb_rusher)')
    df_play = df_play.query('~game_play_id.isin(@gpids_with_qb_rusher)')
    df_player_play = df_player_play.query('~game_play_id.isin(@gpids_with_qb_rusher)')
    df_game = df_game.query('game_id.isin(@gpids_with_qb_rusher)')

Dropping 2 plays with QB rusher


# Verify all plays have 5 labeled oline players (LT, LT, C, RT, RT)

In [8]:
all_gpids = set(df_tracking.game_play_id.unique().tolist())

gpids_with_oline = (
    set(
        df_tracking
        .query('position_by_loc.isin(["LT","LG","C","RG","RT"])')
        .game_play_id
        .unique()
        .tolist()
    )
)

all_gpids - gpids_with_oline

set()

# Create Features for Run Scheme Classification
* need to recreate oline box and label extra players on oline. The features for this created in previous notebooks are at the point of the last line set, not at the point of last line-set event, which could mean there is motion after this before ball-snap. Need to create these features at the time of ball-snap

In [9]:
# cols = [
#     'game_play_id',
#     'rush_location_type','run_location_desc','run_location',
#     'pff_run_concept_primary','pff_run_concept_secondary',
#     'offense_personnel','offense_formation','receiver_alignment','play_action','pff_run_pass_option',
#     'defense_personnel','defenders_in_box'
# ]
# df_play[cols].head(40)

In [10]:
cols = ['game_id', 'play_id', 'game_play_id', 'nfl_id', 'week', 'display_name',
       'frame_id', 'frame_type', 'time', 'jersey_number', 'club',
       'play_direction', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event',
       'position', 'absolute_yardline_number', 'yards_to_go', 'offense',
       'defense', 'ball_x', 'ball_y', 'event_new', 'position_by_loc', 'motion_player',
       'ball_snap_fid']
df_tracking = (
    df_tracking[cols]
    .merge(
        df_player_play[['game_play_id','nfl_id','had_rush_attempt']],
        on=['game_play_id','nfl_id'],
        how='left'
    )
)

## a. offensive line box features

In [11]:
# Feature of the min and max x values of the 5 offensive linemen at the snap
oline_at_snap = (
    df_tracking
    .query('frame_id == ball_snap_fid and position_by_loc.isin(["LT","LG","C","RG","RT"])')
    [['game_play_id','position_by_loc','x','y']]
)
oline_x_min = oline_at_snap.groupby('game_play_id').x.min().reset_index().rename(columns={'x':'oline_x_left_at_snap'})
oline_x_max = oline_at_snap.groupby('game_play_id').x.max().reset_index().rename(columns={'x':'oline_x_right_at_snap'})
oline_y_min_right = (
    oline_at_snap
    .query('position_by_loc.isin(["C","RG","RT"])')
    .groupby('game_play_id')
    .y.min()
    .reset_index()
    .rename(columns={'y':'oline_y_min_right_at_snap'})
)
oline_y_min_left = (
    oline_at_snap
    .query('position_by_loc.isin(["LT","LG","C"])')
    .groupby('game_play_id')
    .y.min()
    .reset_index()
    .rename(columns={'y':'oline_y_min_left_at_snap'})
)
if 'oline_x_left_at_snap' in df_tracking.columns:
    df_tracking.drop(columns=['oline_x_left_at_snap'], inplace=True)
if 'oline_x_right_at_snap' in df_tracking.columns:
    df_tracking.drop(columns=['oline_x_right_at_snap'], inplace=True)
if 'oline_y_min_left_at_snap' in df_tracking.columns:
    df_tracking.drop(columns=['oline_y_min_left_at_snap'], inplace=True)
if 'oline_y_min_right_at_snap' in df_tracking.columns:
    df_tracking.drop(columns=['oline_y_min_right_at_snap'], inplace=True)
df_tracking = (
    df_tracking
    .merge(oline_x_min, on='game_play_id', how='left')
    .merge(oline_x_max, on='game_play_id', how='left')
    .merge(oline_y_min_left, on='game_play_id', how='left')
    .merge(oline_y_min_right, on='game_play_id', how='left')
)
del oline_at_snap, oline_x_min, oline_x_max, oline_y_min_left, oline_y_min_right

# Feature of Center x at ball snap
center_at_snap = (
    df_tracking
    .query('frame_id == ball_snap_fid and position_by_loc == "C"')
    [['game_play_id','x']]
    .rename(columns={'x':'center_x_at_snap'})
)
if 'center_x_at_snap' in df_tracking.columns:
    df_tracking.drop(columns='center_x_at_snap', inplace=True)
df_tracking = df_tracking.merge(center_at_snap, on='game_play_id', how='left')
del center_at_snap

# Label Extra Players acting as part of the OLine
MAX_YARDS_BEHIND_OLINE = 1
MAX_YARDS_NEXT_TO_OLINE = 2
MOVING_S_THRESHOLD = 1
df_tracking['extra_on_oline_x_left_at_snap'] = df_tracking.oline_x_left_at_snap
df_tracking['extra_on_oline_x_right_at_snap'] = df_tracking.oline_x_right_at_snap
df_tracking['extra_on_oline_y_min_left_at_snap'] = df_tracking.oline_y_min_left_at_snap
df_tracking['extra_on_oline_y_min_right_at_snap'] = df_tracking.oline_y_min_right_at_snap
df_tracking['on_oline'] = np.where(
    df_tracking.position_by_loc.isin(["LT","LG","C","RG","RT"]),
    True,
    False
)
extra_on_oline = pd.DataFrame()
extra_on_oline_len = 1
while extra_on_oline_len != len(extra_on_oline):
    extra_on_oline = (
        df_tracking
        .query('(position_by_loc != "QB") and ~on_oline and offense and frame_id == ball_snap_fid')
        [['game_play_id','nfl_id','x','y','s',
        'extra_on_oline_y_min_left_at_snap','extra_on_oline_y_min_right_at_snap',
        'extra_on_oline_x_left_at_snap','extra_on_oline_x_right_at_snap','on_oline']]
    )
    
    # Save initial length to check if we need to run the loop again
    extra_on_oline_len = len(extra_on_oline)

    extra_on_oline['on_oline'] = np.where(
        (extra_on_oline.s < MOVING_S_THRESHOLD) &
        (
            (
                (extra_on_oline.x < extra_on_oline.extra_on_oline_x_left_at_snap) &
                (extra_on_oline.x >= extra_on_oline.extra_on_oline_x_left_at_snap - MAX_YARDS_NEXT_TO_OLINE) &
                (extra_on_oline.y >= extra_on_oline.extra_on_oline_y_min_left_at_snap - MAX_YARDS_BEHIND_OLINE)
            ) |
            (
                (extra_on_oline.x > extra_on_oline.extra_on_oline_x_right_at_snap) &
                (extra_on_oline.x <= extra_on_oline.extra_on_oline_x_right_at_snap + MAX_YARDS_NEXT_TO_OLINE) &
                (extra_on_oline.y >= extra_on_oline.extra_on_oline_y_min_right_at_snap - MAX_YARDS_BEHIND_OLINE)
            )
        ),
        True,
        False
    )

    # Update oline box left for the new players on the oline left side
    extra_on_online_x_min = (
        extra_on_oline
        .query('on_oline')
        [['game_play_id','nfl_id','x']]
        .groupby('game_play_id')
        .x.min()
        .reset_index()
        .rename(columns={'x':'extra_on_oline_x_left_at_snap'})
    )
    df_tracking = (
        df_tracking
        .merge(
            extra_on_online_x_min[['game_play_id','extra_on_oline_x_left_at_snap']], 
            on='game_play_id', 
            how='left', 
            suffixes=('','_new')
        )
        .assign(
            extra_on_oline_x_left_at_snap = lambda x: np.where(
                x.extra_on_oline_x_left_at_snap_new.notnull() &
                (x.extra_on_oline_x_left_at_snap_new < x.extra_on_oline_x_left_at_snap),
                x.extra_on_oline_x_left_at_snap_new,
                x.extra_on_oline_x_left_at_snap
            )
        )
        .drop(columns='extra_on_oline_x_left_at_snap_new')
    )

    # Update oline box right for the new players on the oline right side
    extra_on_online_x_max = (
        extra_on_oline
        .query('on_oline')
        [['game_play_id','nfl_id','x']]
        .groupby('game_play_id')
        .x.max()
        .reset_index()
        .rename(columns={'x':'extra_on_oline_x_right_at_snap'})
    )
    df_tracking = (
        df_tracking
        .merge(
            extra_on_online_x_max[['game_play_id','extra_on_oline_x_right_at_snap']], 
            on='game_play_id', 
            how='left', 
            suffixes=('','_new')
        )
        .assign(
            extra_on_oline_x_right_at_snap = lambda x: np.where(
                x.extra_on_oline_x_right_at_snap_new.notnull() &
                (x.extra_on_oline_x_right_at_snap_new > x.extra_on_oline_x_right_at_snap),
                x.extra_on_oline_x_right_at_snap_new,
                x.extra_on_oline_x_right_at_snap
            )
        )
        .drop(columns='extra_on_oline_x_right_at_snap_new')
    )

    # Update on_oline for the new players on the oline
    on_oline = extra_on_oline.query('on_oline')[['game_play_id','nfl_id','on_oline']]
    df_tracking = (
        df_tracking.merge(
            on_oline,
            on=['game_play_id','nfl_id'],
            how='left',
            suffixes=('','_new')
        )
        .assign(
            on_oline = lambda x: np.where(
                x.on_oline_new.notnull(),
                x.on_oline_new,
                x.on_oline
            )
        )
        .drop(columns='on_oline_new')
    )

    oline_y_min_left = (
        df_tracking
        .query('on_oline and x < center_x_at_snap')
        [['game_play_id','y']]
        .groupby('game_play_id')
        .y.min()
        .reset_index()
        .rename(columns={'y':'extra_on_oline_y_min_left_at_snap'})
    )
    oline_y_min_right = (
        df_tracking
        .query('on_oline and x > center_x_at_snap')
        [['game_play_id','y']]
        .groupby('game_play_id')
        .y.min()
        .reset_index()
        .rename(columns={'y':'extra_on_oline_y_min_right_at_snap'})
    )
    df_tracking = (
        df_tracking
        .drop(columns=['extra_on_oline_y_min_left_at_snap','extra_on_oline_y_min_right_at_snap'])
        .merge(oline_y_min_left, on='game_play_id', how='left')
        .merge(oline_y_min_right, on='game_play_id', how='left')
    )

    extra_on_oline = extra_on_oline.query('~on_oline')
    del extra_on_online_x_min, extra_on_online_x_max, on_oline, oline_y_min_left, oline_y_min_right

del extra_on_oline

## b. offensive line movement angle feature

In [12]:
oline_xy_at_snap = (
    df_tracking
    .query('frame_id == ball_snap_fid and position_by_loc.isin(["LT","LG","C","RG","RT"])')
    [['game_play_id','nfl_id','x','y']]
    .rename(columns={'x':'oline_x_at_snap','y':'oline_y_at_snap'})
)
if 'oline_x_at_snap' in df_tracking.columns:
    df_tracking.drop(columns=['oline_x_at_snap'], inplace=True)
if 'oline_y_at_snap' in df_tracking.columns:
    df_tracking.drop(columns=['oline_y_at_snap'], inplace=True)
df_tracking = df_tracking.merge(oline_xy_at_snap, on=['game_play_id','nfl_id'], how='left')
del oline_xy_at_snap

oline_xy_1s_after_snap = (
    df_tracking
    .query('frame_id == ball_snap_fid + 10 and position_by_loc.isin(["LT","LG","C","RG","RT"])')
    [['game_play_id','nfl_id','x','y']]
    .rename(columns={'x':'oline_x_1s_after_snap','y':'oline_y_1s_after_snap'})
)
if 'oline_x_1s_after_snap' in df_tracking.columns:
    df_tracking.drop(columns=['oline_x_1s_after_snap'], inplace=True)
if 'oline_y_1s_after_snap' in df_tracking.columns:
    df_tracking.drop(columns=['oline_y_1s_after_snap'], inplace=True)
df_tracking = df_tracking.merge(oline_xy_1s_after_snap, on=['game_play_id','nfl_id'], how='left')
del oline_xy_1s_after_snap

df_tracking['dy_oline_1s_after_snap'] = df_tracking.oline_y_1s_after_snap - df_tracking.oline_y_at_snap
df_tracking['dx_oline_1s_after_snap'] = df_tracking.oline_x_1s_after_snap - df_tracking.oline_x_at_snap
df_tracking['oline_angle_1s_after_snap'] = np.degrees(np.arctan2(
    df_tracking.dy_oline_1s_after_snap, 
    df_tracking.dx_oline_1s_after_snap
))
df_tracking['oline_angle_1s_after_snap'] = np.where(
    df_tracking.oline_angle_1s_after_snap < 0,
    360 + df_tracking.oline_angle_1s_after_snap,
    df_tracking.oline_angle_1s_after_snap
)

In [13]:
cols = [
    'game_play_id',
    'rush_location_type','run_location_desc','run_location',
    'pff_run_concept_primary','pff_run_concept_secondary',
    'offense_personnel','offense_formation','receiver_alignment','play_action','pff_run_pass_option',
    'defense_personnel','defenders_in_box'
]
df_play[cols].head(40)

,game_play_id,rush_location_type,run_location_desc,run_location,pff_run_concept_primary,pff_run_concept_secondary,offense_personnel,offense_formation,receiver_alignment,play_action,pff_run_pass_option,defense_personnel,defenders_in_box
0,2022091806_346,OUTSIDE_RIGHT,middle,middle,MAN,NaN,"1 RB, 1 TE, 3 WR",SHOTGUN,2x2,False,0,"3 DL, 3 LB, 5 DB",6.0
1,2022091804_699,INSIDE_RIGHT,left_tackle,left,OUTSIDE ZONE,NaN,"1 RB, 1 TE, 3 WR",SINGLEBACK,2x2,False,0,"2 DL, 4 LB, 5 DB",7.0
2,2022091811_120,INSIDE_RIGHT,middle,middle,TRAP,LEAD,"2 RB, 1 TE, 2 WR",I_FORM,2x1,False,0,"3 DL, 4 LB, 4 DB",7.0
3,2022091806_950,INSIDE_RIGHT,right_tackle,right,OUTSIDE ZONE,NaN,"1 RB, 3 TE, 1 WR",SINGLEBACK,2x2,False,0,"3 DL, 3 LB, 5 DB",8.0
4,2022091810_3528,OUTSIDE_RIGHT,middle,middle,MAN,NaN,"1 RB, 1 TE, 3 WR",SINGLEBACK,2x2,False,0,"4 DL, 2 LB, 5 DB",6.0
5,2022091804_2966,INSIDE_LEFT,left_guard,left,MAN,NaN,"1 RB, 2 TE, 2 WR",SINGLEBACK,3x1,False,0,"5 DL, 2 LB, 4 DB",8.0
6,2022091807_1196,OUTSIDE_RIGHT,right_tackle,right,OUTSIDE ZONE,NaN,"2 RB, 2 TE, 1 WR",SINGLEBACK,2x2,False,0,"3 DL, 4 LB, 4 DB",8.0
7,2022091804_4294,OUTSIDE_RIGHT,left_guard,left,OUTSIDE ZONE,NaN,"1 RB, 2 TE, 2 WR",SINGLEBACK,3x1,False,0,"4 DL, 3 LB, 4 DB",7.0
8,2022091811_141,INSIDE_LEFT,left_guard,left,COUNTER,CROSS LEAD,"2 RB, 1 TE, 2 WR",SHOTGUN,2x1,False,0,"3 DL, 4 LB, 4 DB",7.0
9,2022091809_122,INSIDE_RIGHT,left_tackle,left,OUTSIDE ZONE,NaN,"1 RB, 2 TE, 2 WR",SINGLEBACK,3x1,False,0,"5 DL, 2 LB, 4 DB",7.0


In [35]:
gpid = "2022091500_1279"
(
    df_tracking
    .query('frame_id == ball_snap_fid and position_by_loc.isin(["LT","LG","C","RG","RT"]) and game_play_id == @gpid')
    [['game_play_id','position_by_loc','x','y','oline_angle_1s_after_snap']]
    .sort_values('x')
)

,game_play_id,position_by_loc,x,y,oline_angle_1s_after_snap
1982635,2022091500_1279,LT,20.29,57.65,129.126038
1980269,2022091500_1279,LG,21.81,57.86,192.407419
1980087,2022091500_1279,C,23.49,58.55,183.865461
1983363,2022091500_1279,RG,25.08,58.09,165.550970
1981725,2022091500_1279,RT,26.70,57.93,325.304846


In [45]:
(
    df_tracking
    .drop_duplicates(['game_play_id','nfl_id'])
    .merge(
        (
            df_play
            [['game_play_id','pff_run_concept_primary','rush_location_type']]
            .rename(columns={'pff_run_concept_primary':'run_concept'})
        ),
        on='game_play_id',
        how='left'
    )
    .query('run_concept == "OUTSIDE ZONE"')
    .query('position_by_loc.isin(["LT","LG","C","RG","RT"])')
    .query('80 <= oline_angle_1s_after_snap <= 100')
    [['game_play_id','position_by_loc','x','y','oline_angle_1s_after_snap','rush_location_type']]
    .head(10)
)

,game_play_id,position_by_loc,x,y,oline_angle_1s_after_snap,rush_location_type
3619,2022091810_1272,RG,27.73,47.23,81.158185,OUTSIDE_RIGHT
5001,2022091809_3158,LT,20.62,41.71,89.632724,OUTSIDE_LEFT
6061,2022091807_533,LG,29.33,96.46,98.829745,INSIDE_RIGHT


In [61]:
cols = [
    'game_play_id',
    'rush_location_type','run_location_desc','run_location',
    'pff_run_concept_primary','pff_run_concept_secondary',
    'offense_personnel','offense_formation','receiver_alignment','play_action','pff_run_pass_option',
    'defense_personnel','defenders_in_box'
]
df_play[cols].query('pff_run_concept_primary=="POWER"').head(40)

,game_play_id,rush_location_type,run_location_desc,run_location,pff_run_concept_primary,pff_run_concept_secondary,offense_personnel,offense_formation,receiver_alignment,play_action,pff_run_pass_option,defense_personnel,defenders_in_box
12,2022091801_2635,INSIDE_LEFT,left_guard,left,POWER,LEAD,"1 RB, 2 TE, 2 WR",SINGLEBACK,2x2,False,0,"4 DL, 3 LB, 4 DB",8.0
18,2022091805_954,OUTSIDE_LEFT,left_end,left,POWER,NaN,"1 RB, 1 TE, 3 WR",SHOTGUN,2x2,False,1,"3 DL, 2 LB, 6 DB",7.0
29,2022091804_2826,OUTSIDE_LEFT,left_guard,left,POWER,NaN,"1 RB, 2 TE, 2 WR",SINGLEBACK,2x2,False,0,"4 DL, 3 LB, 4 DB",7.0
30,2022091802_269,OUTSIDE_LEFT,left_end,left,POWER,NaN,"1 RB, 1 TE, 3 WR",SINGLEBACK,3x1,False,0,"5 DL, 1 LB, 5 DB",6.0
61,2022091808_2670,INSIDE_LEFT,middle,middle,POWER,LEAD,"2 RB, 1 TE, 2 WR",I_FORM,2x1,False,0,"4 DL, 3 LB, 4 DB",8.0
68,2022091806_2013,INSIDE_RIGHT,left_guard,left,POWER,NaN,"1 RB, 1 TE, 3 WR",SHOTGUN,2x2,False,0,"3 DL, 3 LB, 5 DB",6.0
99,2022091812_769,OUTSIDE_RIGHT,right_tackle,right,POWER,LEAD,"1 RB, 2 TE, 2 WR",I_FORM,2x1,False,0,"3 DL, 4 LB, 4 DB",7.0
132,2022091812_2620,OUTSIDE_LEFT,left_guard,left,POWER,LEAD,"2 RB, 1 TE, 2 WR",SINGLEBACK,2x2,False,0,"3 DL, 3 LB, 5 DB",6.0
134,2022091901_2126,INSIDE_LEFT,left_guard,left,POWER,NaN,"1 RB, 1 TE, 3 WR",SHOTGUN,2x2,False,0,"4 DL, 2 LB, 5 DB",7.0
137,2022091811_56,INSIDE_LEFT,middle,middle,POWER,LEAD,"2 RB, 2 TE, 1 WR",I_FORM,2x1,False,0,"3 DL, 4 LB, 4 DB",7.0


In [32]:
df_run_concepts = pd.DataFrame()
for wk in tqdm(range(1,10)):
    df_trk_tmp = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{wk}', 'tracking_final.pkl'))
    df_ply_tmp = pd.read_pickle(join(PROCESSED_DATA_PATH, f'wk{wk}', 'play_final.pkl'))[['game_play_id','pff_run_concept_primary']]
    df_trk_tmp = (
        df_trk_tmp
        [['game_play_id','motion_player']]
        .sort_values('motion_player', ascending=False)
        .drop_duplicates(['game_play_id'], keep='first')
        .merge(
            (
                df_ply_tmp
                [['game_play_id','pff_run_concept_primary']]
                .rename(columns={'pff_run_concept_primary':'run_concept'})
            ),
            on='game_play_id',
            how='left'
        )
    )
    df_run_concepts = pd.concat([df_run_concepts, df_trk_tmp])
del df_trk_tmp
df_run_concepts.value_counts(['run_concept','motion_player'])

KeyboardInterrupt: 

In [31]:
(
    df_tracking
    .sort_values('motion_player', ascending=False)
    .drop_duplicates(['game_play_id'], keep='first')
    .merge(
        (
            df_play
            [['game_play_id','pff_run_concept_primary','rush_location_type']]
            .rename(columns={'pff_run_concept_primary':'run_concept'})
        ),
        on='game_play_id',
        how='left'
    )
    .value_counts(['run_concept','motion_player'])
)

run_concept   motion_player
OUTSIDE ZONE  False            108
              True              88
INSIDE ZONE   False             77
MAN           False             55
              True              36
INSIDE ZONE   True              34
POWER         False             30
PULL LEAD     True              21
              False             20
COUNTER       False             20
POWER         True              18
TRICK         True              18
COUNTER       True              15
DRAW          False             10
TRAP          True               8
              False              7
FB RUN        False              3
TRICK         False              3
FB RUN        True               2
UNDEFINED     False              1
dtype: int64

In [25]:
df_tracking.sort_values('motion_player', ascending=False).motion_player.head()

1226868    True
912273     True
912265     True
912266     True
912267     True
Name: motion_player, dtype: bool

In [22]:
df_tracking.head()

,game_id,play_id,game_play_id,nfl_id,week,display_name,frame_id,frame_type,time,jersey_number,club,play_direction,x,y,s,a,dis,o,dir,event,position,absolute_yardline_number,yards_to_go,offense,defense,ball_x,ball_y,event_new,position_by_loc,motion_player,ball_snap_fid,had_rush_attempt,oline_x_left_at_snap,oline_x_right_at_snap,oline_y_min_left_at_snap,oline_y_min_right_at_snap,center_x_at_snap,extra_on_oline_x_left_at_snap,extra_on_oline_x_right_at_snap,on_oline,extra_on_oline_y_min_left_at_snap,extra_on_oline_y_min_right_at_snap,oline_x_at_snap,oline_y_at_snap,oline_x_1s_after_snap,oline_y_1s_after_snap,dy_oline_1s_after_snap,dx_oline_1s_after_snap,oline_angle_1s_after_snap
0,2022091901,112,2022091901_112,37078.0,2,Patrick Peterson,1,BEFORE_SNAP,2022-09-20 00:33:10.7,7.0,MIN,right,44.24,57.14,0.66,0.46,0.07,185.48,11.51,NaN,CB,51,10,False,True,29.590001,50.730000,NaN,CB,False,63,0.0,26.27,32.68,49.26,49.28,29.54,26.27,33.98,False,46.37,46.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022091901,112,2022091901_112,37078.0,2,Patrick Peterson,2,BEFORE_SNAP,2022-09-20 00:33:10.8,7.0,MIN,right,44.30,57.15,0.63,0.46,0.06,186.59,7.07,NaN,CB,51,10,False,True,29.590001,50.720001,NaN,CB,False,63,0.0,26.27,32.68,49.26,49.28,29.54,26.27,33.98,False,46.37,46.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022091901,112,2022091901_112,37078.0,2,Patrick Peterson,3,BEFORE_SNAP,2022-09-20 00:33:10.9,7.0,MIN,right,44.37,57.15,0.61,0.41,0.06,186.59,3.82,NaN,CB,51,10,False,True,29.580001,50.720001,NaN,CB,False,63,0.0,26.27,32.68,49.26,49.28,29.54,26.27,33.98,False,46.37,46.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022091901,112,2022091901_112,37078.0,2,Patrick Peterson,4,BEFORE_SNAP,2022-09-20 00:33:11,7.0,MIN,right,44.42,57.16,0.54,0.39,0.05,185.68,4.33,NaN,CB,51,10,False,True,29.580001,50.720001,NaN,CB,False,63,0.0,26.27,32.68,49.26,49.28,29.54,26.27,33.98,False,46.37,46.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022091901,112,2022091901_112,37078.0,2,Patrick Peterson,5,BEFORE_SNAP,2022-09-20 00:33:11.1,7.0,MIN,right,44.47,57.16,0.48,0.51,0.05,183.80,1.59,NaN,CB,51,10,False,True,29.580001,50.720001,NaN,CB,False,63,0.0,26.27,32.68,49.26,49.28,29.54,26.27,33.98,False,46.37,46.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
gpid = "2022091806_2058"

In [17]:
plot_play_with_speed(
    df_tracking.query('ball_snap_fid <= frame_id <= ball_snap_fid + 15 and game_play_id==@gpid'),
    gpid, 
    every_other_frame=True, 
    event_col='event_new',
    plot_motion=False,
    show_motion_frames=False,
    highlight_offensive_positions=True,
    highlight_oline=True,
)